#Homework 2 - Spark

## Lettura dei dati

In [0]:
import requests

from pyspark.sql.functions import when, count, isnull, expr, current_date, col, date_format, year, month, lag, sum, round, max, min, datediff, explode
from pyspark.sql.window import Window

In [0]:
local_path_popolazione_regioni = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/popolazione_regioni.csv"
local_path_dati_covid_regioni = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/dati_covid_regioni.csv"
local_path_dati_vaccini_somministrazioni = "dbfs:/FileStore/shared_uploads/gu.dichiara@studenti.unina.it/dati_vaccini_somministrazioni.csv"

# local_path_popolazione_regioni = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/popolazione_regioni.csv"
# local_path_dati_covid_regioni = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/dati_covid_regioni.csv"
# local_path_dati_vaccini_somministrazioni = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/dati_vaccini_somministrazioni.csv"
# local_path_dati_vaccini_consegne = "dbfs:/FileStore/shared_uploads/fabio.dandrea@studenti.unina.it/dati_vaccini_consegne.csv"

popolazione_regioni_URL = "https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-statistici-riferimento/popolazione-istat-regione-range.csv"
dati_covid_regioni_URL = "https://raw.githubusercontent.com/pcm-dpc/COVID-19/master/dati-regioni/dpc-covid19-ita-regioni.csv"
dati_vaccini_somministrazioni_URL = "https://raw.githubusercontent.com/italia/covid19-opendata-vaccini/master/dati/somministrazioni-vaccini-latest.csv"

dbutils.fs.put(local_path_popolazione_regioni, requests.get(popolazione_regioni_URL).text, True)
dbutils.fs.put(local_path_dati_covid_regioni, requests.get(dati_covid_regioni_URL).text, True)
dbutils.fs.put(local_path_dati_vaccini_somministrazioni, requests.get(dati_vaccini_somministrazioni_URL).text, True)

popolazione_regioni = spark.read.option("inferSchema", "true") \
                                .option("header", "true") \
                                .csv(local_path_popolazione_regioni)
dati_covid_regioni = spark.read.option("inferSchema", "true") \
                                .option("header", "true") \
                                .csv(local_path_dati_covid_regioni)
dati_vaccini_somministrazioni = spark.read.option("inferSchema", "true") \
                                          .option("header", "true") \
                                          .csv(local_path_dati_vaccini_somministrazioni)

Wrote 17300 bytes.
Wrote 1505911 bytes.
Wrote 4009521 bytes.

## Preprocessing

### Popolazione Regioni
Si eliminano le colonne che non sono di interesse.

In [0]:
popolazione_regioni = popolazione_regioni.drop('codice_regione', 'codice_nuts_1', 'descrizione_nuts_1', 'sigla_regione',
                                               'latitudine_regione', 'longitudine_regione', 'totale_genere_maschile', 'totale_genere_femminile')
popolazione_regioni.show()

+-------------+---------------------+---------+---------------+
codice_nuts_2|denominazione_regione|range_eta|totale_generale|
+-------------+---------------------+---------+---------------+
 ITF1| Abruzzo| 0-15| 169897|
 ITF1| Abruzzo| 16-19| 46751|
 ITF1| Abruzzo| 20-29| 130083|
 ITF1| Abruzzo| 30-39| 150713|
 ITF1| Abruzzo| 40-49| 190060|
 ITF1| Abruzzo| 50-59| 204157|
 ITF1| Abruzzo| 60-69| 167705|
 ITF1| Abruzzo| 70-79| 130572|
 ITF1| Abruzzo| 80-89| 84488|
 ITF1| Abruzzo| 90+| 19515|
 ITF5| Basilicata| 0-15| 69994|
 ITF5| Basilicata| 16-19| 22032|
 ITF5| Basilicata| 20-29| 61183|
 ITF5| Basilicata| 30-39| 65204|
 ITF5| Basilicata| 40-49| 78467|
 ITF5| Basilicata| 50-59| 87822|
 ITF5| Basilicata| 60-69| 72817|
 ITF5| Basilicata| 70-79| 51805|
 ITF5| Basilicata| 80-89| 36107|
 ITF5| Basilicata| 90+| 7823|
+-------------+---------------------+---------+---------------+
only showing top 20 rows

Si verifica che il formato dei dati sia corretto.

In [0]:
popolazione_regioni.dtypes

Out[61]: [('codice_nuts_2', 'string'),
 ('denominazione_regione', 'string'),
 ('range_eta', 'string'),
 ('totale_generale', 'int')]

Si verifica la correttezza delle colonne categoriche.

In [0]:
num_regioni = popolazione_regioni.groupBy('denominazione_regione') \
                .count() \
                .count()
print('Il numero di regioni è:', num_regioni)
print()
popolazione_regioni.groupBy('codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .orderBy('codice_nuts_2') \
                    .show(num_regioni)

Il numero di regioni è: 21

+-------------+---------------------+
codice_nuts_2|denominazione_regione|
+-------------+---------------------+
 ITC1| Piemonte|
 ITC2| Valle d'Aosta|
 ITC3| Liguria|
 ITC4| Lombardia|
 ITF1| Abruzzo|
 ITF2| Molise|
 ITF3| Campania|
 ITF4| Puglia|
 ITF5| Basilicata|
 ITF6| Calabria|
 ITG1| Sicilia|
 ITG2| Sardegna|
 ITH1| Bolzano|
 ITH2| Trento|
 ITH3| Veneto|
 ITH4| Friuli Venezia Gi...|
 ITH5| Emilia-Romagna|
 ITI1| Toscana|
 ITI2| Umbria|
 ITI3| Marche|
 ITI4| Lazio|
+-------------+---------------------+

In [0]:
popolazione_regioni.groupBy('range_eta') \
                    .count() \
                    .orderBy('range_eta') \
                    .show()

+---------+-----+
range_eta|count|
+---------+-----+
 0-15| 21|
 16-19| 21|
 20-29| 21|
 30-39| 21|
 40-49| 21|
 50-59| 21|
 60-69| 21|
 70-79| 21|
 80-89| 21|
 90+| 21|
+---------+-----+

Si verifica la correttezza delle colonne numeriche.

In [0]:
popolazione_regioni.describe('totale_generale') \
                   .show()

+-------+-----------------+
summary| totale_generale|
+-------+-----------------+
 count| 210|
 mean|284007.0857142857|
 stddev|303588.4857882117|
 min| 1764|
 max| 1592109|
+-------+-----------------+

Si verifica la presenza di *missing values*.

In [0]:
popolazione_regioni.select([count(when(isnull(c), c)).alias(c) for c in popolazione_regioni.columns]).show()

+-------------+---------------------+---------+---------------+
codice_nuts_2|denominazione_regione|range_eta|totale_generale|
+-------------+---------------------+---------+---------------+
 0| 0| 0| 0|
+-------------+---------------------+---------+---------------+

### Dati Covid Regioni
Si eliminano le colonne che non sono di interesse.

In [0]:
dati_covid_regioni = dati_covid_regioni.drop('stato', 'codice_nuts_1', 'codice_regione', 'lat', 'long', 'note_test', 'note_casi', 'note',
                                             'casi_da_sospetto_diagnostico', 'casi_da_screening',
                                             'casi_testati', 'ingressi_terapia_intensiva', 
                                             'totale_positivi_test_molecolare', 'totale_positivi_test_antigenico_rapido',
                                             'tamponi_test_molecolare', 'tamponi_test_antigenico_rapido', 'isolamento_domiciliare', 'variazione_totale_positivi', 'dimessi_guariti', 'totale_casi')
dati_covid_regioni.show()

+-------------------+---------------------+----------------------+-----------------+--------------------+---------------+--------------+--------+-------+-------------+
 data|denominazione_regione|ricoverati_con_sintomi|terapia_intensiva|totale_ospedalizzati|totale_positivi|nuovi_positivi|deceduti|tamponi|codice_nuts_2|
+-------------------+---------------------+----------------------+-----------------+--------------------+---------------+--------------+--------+-------+-------------+
2020-02-24 18:00:00| Abruzzo| 0| 0| 0| 0| 0| 0| 5| null|
2020-02-24 18:00:00| Basilicata| 0| 0| 0| 0| 0| 0| 0| null|
2020-02-24 18:00:00| Calabria| 0| 0| 0| 0| 0| 0| 1| null|
2020-02-24 18:00:00| Campania| 0| 0| 0| 0| 0| 0| 10| null|
2020-02-24 18:00:00| Emilia-Romagna| 10| 2| 12| 18| 18| 0| 148| null|
2020-02-24 18:00:00| Friuli Venezia Gi...| 0| 0| 0| 0| 0| 0| 58| null|
2020-02-24 18:00:00| Lazio| 1| 1| 2| 2| 2| 0| 124| null|
2020-02-24 18:00:00| Liguria| 0| 0| 0| 0| 0| 0| 1| null|
2020-02-24 18:00:00| Lombardia| 76| 19| 95| 166| 166| 6| 1463| null|
2020-02-24 18:00:00| Marche| 0| 0| 0| 0| 0| 0| 16| null|
2020-02-24 18:00:00| Molise| 0| 0| 0| 0| 0| 0| 0| null|
2020-02-24 18:00:00| P.A. Bolzano| 0| 0| 0| 0| 0| 0| 1| null|
2020-02-24 18:00:00| P.A. Trento| 0| 0| 0| 0| 0| 0| 3| null|
2020-02-24 18:00:00| Piemonte| 2| 0| 2| 3| 3| 0| 141| null|
2020-02-24 18:00:00| Puglia| 0| 0| 0| 0| 0| 0| 0| null|
2020-02-24 18:00:00| Sardegna| 0| 0| 0| 0| 0| 0| 1| null|
2020-02-24 18:00:00| Sicilia| 0| 0| 0| 0| 0| 0| 5| null|
2020-02-24 18:00:00| Toscana| 0| 0| 0| 0| 0| 0| 140| null|
2020-02-24 18:00:00| Umbria| 0| 0| 0| 0| 0| 0| 0| null|
2020-02-24 18:00:00| Valle d'Aosta| 0| 0| 0| 0| 0| 0| 7| null|
+-------------------+---------------------+----------------------+-----------------+--------------------+---------------+--------------+--------+-------+-------------+
only showing top 20 rows

Si verifica che il formato dei dati sia corretto.

In [0]:
dati_covid_regioni.dtypes

Out[67]: [('data', 'timestamp'),
 ('denominazione_regione', 'string'),
 ('ricoverati_con_sintomi', 'int'),
 ('terapia_intensiva', 'int'),
 ('totale_ospedalizzati', 'int'),
 ('totale_positivi', 'int'),
 ('nuovi_positivi', 'int'),
 ('deceduti', 'int'),
 ('tamponi', 'int'),
 ('codice_nuts_2', 'string')]

In [0]:
dati_covid_regioni = dati_covid_regioni.withColumn('data', date_format(dati_covid_regioni.data, 'yyyy-MM-dd').cast('date'))

In [0]:
dati_covid_regioni.dtypes

Out[69]: [('data', 'date'),
 ('denominazione_regione', 'string'),
 ('ricoverati_con_sintomi', 'int'),
 ('terapia_intensiva', 'int'),
 ('totale_ospedalizzati', 'int'),
 ('totale_positivi', 'int'),
 ('nuovi_positivi', 'int'),
 ('deceduti', 'int'),
 ('tamponi', 'int'),
 ('codice_nuts_2', 'string')]

Si verifica la correttezza delle colonne categoriche.

In [0]:
num_regioni = dati_covid_regioni.groupBy('denominazione_regione') \
                .count() \
                .count()
print('Il numero di regioni è:', num_regioni)
print()
dati_covid_regioni.groupBy('codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .orderBy('codice_nuts_2') \
                    .show(50)

Il numero di regioni è: 21

+-------------+---------------------+
codice_nuts_2|denominazione_regione|
+-------------+---------------------+
 null| Lazio|
 null| Sicilia|
 null| Emilia-Romagna|
 null| Liguria|
 null| Toscana|
 null| Puglia|
 null| Abruzzo|
 null| Sardegna|
 null| Umbria|
 null| Piemonte|
 null| P.A. Bolzano|
 null| Molise|
 null| Calabria|
 null| Friuli Venezia Gi...|
 null| Valle d'Aosta|
 null| P.A. Trento|
 null| Marche|
 null| Basilicata|
 null| Veneto|
 null| Campania|
 null| Lombardia|
 ITC1| Piemonte|
 ITC2| Valle d'Aosta|
 ITC3| Liguria|
 ITC4| Lombardia|
 ITF1| Abruzzo|
 ITF2| Molise|
 ITF3| Campania|
 ITF4| Puglia|
 ITF5| Basilicata|
 ITF6| Calabria|
 ITG1| Sicilia|
 ITG2| Sardegna|
 ITH| P.A. Bolzano|
 ITH1| P.A. Bolzano|
 ITH2| P.A. Trento|
 ITH3| Veneto|
 ITH4| Friuli Venezia Gi...|
 ITH5| Emilia-Romagna|
 ITI| Umbria|
 ITI1| Toscana|
 ITI2| Umbria|
 ITI3| Marche|
 ITI4| Lazio|
+-------------+---------------------+

Si possono fare alcune osservazioni:
- per alcune entry mancano i codici nuts;
- per aclune entry i codici nuts sono sbagliati (problema dovuto alla lettura del file csv);
- alcune regioni (Bolzano e Trento) hanno una denominazione diversa rispetto alla tabella `popolazione_regioni`;

In [0]:
join1 = popolazione_regioni.join(dati_covid_regioni, on='denominazione_regione', how='anti') \
                    .groupBy('codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .withColumnRenamed('denominazione_regione', 'popolazione_regioni_denominazione_regione')

join2 = dati_covid_regioni.join(popolazione_regioni, on='denominazione_regione', how='anti') \
                    .groupBy('codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') \
                    .withColumnRenamed('denominazione_regione', 'dati_covid_regioni_denominazione_regione')

join1.join(join2, on='codice_nuts_2').show()

dati_covid_regioni = dati_covid_regioni.replace('P.A. Trento', value='Trento', subset='denominazione_regione')
dati_covid_regioni = dati_covid_regioni.replace('P.A. Bolzano', value='Bolzano', subset='denominazione_regione')

+-------------+-----------------------------------------+----------------------------------------+
codice_nuts_2|popolazione_regioni_denominazione_regione|dati_covid_regioni_denominazione_regione|
+-------------+-----------------------------------------+----------------------------------------+
 ITH1| Bolzano| P.A. Bolzano|
 ITH2| Trento| P.A. Trento|
+-------------+-----------------------------------------+----------------------------------------+

In [0]:
dati_covid_regioni = dati_covid_regioni.drop('codice_nuts_2')

nuts_2 = popolazione_regioni.groupBy('codice_nuts_2', 'denominazione_regione') \
                            .count() \
                            .drop('count')

dati_covid_regioni = dati_covid_regioni.join(nuts_2, on=['denominazione_regione'], how='left')

dati_covid_regioni.groupBy('codice_nuts_2', 'denominazione_regione') \
                  .count() \
                  .drop('count') \
                  .orderBy('codice_nuts_2').show(num_regioni)

+-------------+---------------------+
codice_nuts_2|denominazione_regione|
+-------------+---------------------+
 ITC1| Piemonte|
 ITC2| Valle d'Aosta|
 ITC3| Liguria|
 ITC4| Lombardia|
 ITF1| Abruzzo|
 ITF2| Molise|
 ITF3| Campania|
 ITF4| Puglia|
 ITF5| Basilicata|
 ITF6| Calabria|
 ITG1| Sicilia|
 ITG2| Sardegna|
 ITH1| Bolzano|
 ITH2| Trento|
 ITH3| Veneto|
 ITH4| Friuli Venezia Gi...|
 ITH5| Emilia-Romagna|
 ITI1| Toscana|
 ITI2| Umbria|
 ITI3| Marche|
 ITI4| Lazio|
+-------------+---------------------+

Si verifica la correttezza delle colonne numeriche.

In [0]:
dati_covid_regioni.describe(['ricoverati_con_sintomi', 'terapia_intensiva', 'totale_ospedalizzati', 'totale_positivi', 'nuovi_positivi', 'deceduti', 'tamponi']) \
                  .show()

+-------+----------------------+-----------------+--------------------+------------------+-----------------+------------------+------------------+
summary|ricoverati_con_sintomi|terapia_intensiva|totale_ospedalizzati| totale_positivi| nuovi_positivi| deceduti| tamponi|
+-------+----------------------+-----------------+--------------------+------------------+-----------------+------------------+------------------+
 count| 9072| 9072| 9072| 9072| 9072| 9072| 9072|
 mean| 712.4712301587301|81.71847442680776| 794.1897045855379|12468.846560846561|442.7234347442681|2470.1935626102295| 848671.4441137566|
 stddev| 1289.9373804295726|143.0112526568166| 1428.1841519339673| 22237.61213787529| 853.939588083822| 4695.206250550821|1338368.1129099485|
 min| 0| 0| 0| 0| -229| 0| 0|
 max| 12077| 1381| 13328| 164406| 11489| 32870| 9475296|
+-------+----------------------+-----------------+--------------------+------------------+-----------------+------------------+------------------+

In [0]:
dati_covid_regioni.filter(condition='nuovi_positivi < 0').show()

+---------------------+----------+----------------------+-----------------+--------------------+---------------+--------------+--------+-------+-------------+
denominazione_regione| data|ricoverati_con_sintomi|terapia_intensiva|totale_ospedalizzati|totale_positivi|nuovi_positivi|deceduti|tamponi|codice_nuts_2|
+---------------------+----------+----------------------+-----------------+--------------------+---------------+--------------+--------+-------+-------------+
 Piemonte|2020-02-27| 2| 0| 2| 2| -1| 0| 156| ITC1|
 Liguria|2020-03-01| 12| 1| 13| 21| -17| 0| 121| ITC3|
 Liguria|2020-03-02| 12| 1| 13| 18| -3| 0| 121| ITC3|
 Sicilia|2020-03-02| 2| 0| 2| 5| -2| 0| 307| ITG1|
 Piemonte|2020-03-09| 222| 50| 272| 337| -10| 13| 1681| ITC1|
 Calabria|2020-04-17| 154| 7| 161| 819| -18| 73| 21657| ITF6|
 Sardegna|2020-05-04| 91| 9| 100| 653| -2| 119| 28052| ITG2|
 Basilicata|2020-05-07| 48| 2| 50| 155| -16| 26| 16272| ITF5|
 Basilicata|2020-05-08| 48| 2| 50| 152| -1| 26| 16777| ITF5|
 Marche|2020-05-19| 144| 17| 161| 2128| -3| 986| 89985| ITI3|
 Sardegna|2020-05-25| 49| 3| 52| 231| -2| 129| 51073| ITG2|
 Sardegna|2020-06-09| 9| 1| 10| 54| -1| 131| 64272| ITG2|
 Campania|2020-06-12| 61| 2| 63| 346| -229| 430| 230551| ITF3|
+---------------------+----------+----------------------+-----------------+--------------------+---------------+--------------+--------+-------+-------------+

Si verifica la presenza di *missing values*.

In [0]:
dati_covid_regioni.select([count(when(isnull(c), c)).alias(c) for c in dati_covid_regioni.drop('data').columns]).show()

+---------------------+----------------------+-----------------+--------------------+---------------+--------------+--------+-------+-------------+
denominazione_regione|ricoverati_con_sintomi|terapia_intensiva|totale_ospedalizzati|totale_positivi|nuovi_positivi|deceduti|tamponi|codice_nuts_2|
+---------------------+----------------------+-----------------+--------------------+---------------+--------------+--------+-------+-------------+
 0| 0| 0| 0| 0| 0| 0| 0| 0|
+---------------------+----------------------+-----------------+--------------------+---------------+--------------+--------+-------+-------------+

Si verifica la presenza di *missing values* nella serie temporale.

In [0]:
min_data = dati_covid_regioni.agg(min('data').alias('minimo'))
max_data = dati_covid_regioni.agg(max('data').alias('massimo'))

date = min_data.join(max_data)
# date = date.withColumn('minimo', date_format(date.minimo, 'yyyy-MM-dd').cast('date')) \
#            .withColumn('massimo', date_format(date.massimo, 'yyyy-MM-dd').cast('date'))

date.show()

+----------+----------+
 minimo| massimo|
+----------+----------+
2020-02-24|2021-04-30|
+----------+----------+

In [0]:
time_series = date.withColumn('data', explode(expr('sequence(minimo, massimo, interval 1 day)'))) \
                  .select('data')

time_series.show()

+----------+
 data|
+----------+
2020-02-24|
2020-02-25|
2020-02-26|
2020-02-27|
2020-02-28|
2020-02-29|
2020-03-01|
2020-03-02|
2020-03-03|
2020-03-04|
2020-03-05|
2020-03-06|
2020-03-07|
2020-03-08|
2020-03-09|
2020-03-10|
2020-03-11|
2020-03-12|
2020-03-13|
2020-03-14|
+----------+
only showing top 20 rows

In [0]:
date_covid = dati_covid_regioni.select('data') \
                               .distinct() \
                               .orderBy('data')

date_covid.join(time_series, on='data', how='anti').show()

+----+
data|
+----+
+----+

### Dati Somministrazioni Vaccini

Si elmininano le colonne che non sono di interesse

In [0]:
dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.drop('area', 'codice_regione_ISTAT', 'sesso_maschile', 'sesso_femminile' 'codice_NUTS1')
dati_vaccini_somministrazioni.show()

+---------------------+---------------+-----------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+----------+
data_somministrazione| fornitore|fascia_anagrafica|sesso_femminile|categoria_operatori_sanitari_sociosanitari|categoria_personale_non_sanitario|categoria_ospiti_rsa|categoria_60_69|categoria_70_79|categoria_over80|categoria_forze_armate|categoria_personale_scolastico|categoria_soggetti_fragili|categoria_altro|prima_dose|seconda_dose|codice_NUTS2| nome_area|
+---------------------+---------------+-----------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+----------+
 2020-12-27|Pfizer/BioNTech| 20-29| 0| 1| 0| 0| 0| 0| 0| 0| 0| 0| 0| 1| 0| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 30-39| 3| 4| 0| 0| 0| 0| 0| 0| 0| 0| 0| 4| 0| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 40-49| 6| 7| 0| 0| 0| 0| 0| 0| 0| 0| 0| 7| 0| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 50-59| 5| 9| 0| 0| 0| 0| 0| 0| 0| 0| 0| 9| 0| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 60-69| 4| 13| 1| 0| 0| 0| 0| 0| 0| 0| 0| 14| 0| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 70-79| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 1| 0| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 80-89| 0| 0| 0| 1| 0| 0| 0| 0| 0| 0| 0| 1| 0| ITF1| Abruzzo|
 2020-12-27|Pfizer/BioNTech| 20-29| 5| 9| 0| 0| 0| 0| 0| 0| 0| 0| 0| 9| 0| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 30-39| 18| 27| 0| 0| 0| 0| 0| 0| 0| 0| 1| 28| 0| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 40-49| 23| 29| 0| 0| 0| 0| 0| 0| 0| 0| 1| 30| 0| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 50-59| 15| 26| 0| 0| 0| 0| 0| 0| 0| 0| 0| 26| 0| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 60-69| 4| 13| 0| 0| 0| 0| 0| 0| 0| 0| 0| 13| 0| ITF5|Basilicata|
 2020-12-27|Pfizer/BioNTech| 20-29| 34| 49| 0| 0| 0| 0| 0| 0| 0| 0| 0| 49| 0| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 30-39| 20| 43| 0| 0| 0| 0| 0| 0| 0| 0| 0| 43| 0| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 40-49| 27| 50| 0| 0| 0| 0| 0| 0| 0| 0| 0| 50| 0| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 50-59| 28| 76| 1| 0| 0| 0| 0| 0| 0| 0| 0| 77| 0| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 60-69| 16| 60| 0| 0| 0| 0| 0| 0| 0| 0| 0| 60| 0| ITF6| Calabria|
 2020-12-27|Pfizer/BioNTech| 20-29| 33| 59| 1| 0| 0| 0| 0| 0| 0| 0| 0| 60| 0| ITF3| Campania|
 2020-12-27|Pfizer/BioNTech| 30-39| 64| 122| 2| 0| 0| 0| 0| 0| 0| 0| 1| 125| 0| ITF3| Campania|
 2020-12-27|Pfizer/BioNTech| 40-49| 87| 164| 3| 0| 0| 0| 0| 0| 0| 0| 0| 167| 0| ITF3| Campania|
+---------------------+---------------+-----------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+----------+
only showing top 20 rows

Si verifica che il formato dei dati sia corretto

In [0]:
dati_vaccini_somministrazioni.dtypes

Out[86]: [('data_somministrazione', 'date'),
 ('fornitore', 'string'),
 ('fascia_anagrafica', 'string'),
 ('sesso_femminile', 'int'),
 ('categoria_operatori_sanitari_sociosanitari', 'int'),
 ('categoria_personale_non_sanitario', 'int'),
 ('categoria_ospiti_rsa', 'int'),
 ('categoria_60_69', 'int'),
 ('categoria_70_79', 'int'),
 ('categoria_over80', 'int'),
 ('categoria_forze_armate', 'int'),
 ('categoria_personale_scolastico', 'int'),
 ('categoria_soggetti_fragili', 'int'),
 ('categoria_altro', 'int'),
 ('prima_dose', 'int'),
 ('seconda_dose', 'int'),
 ('codice_NUTS2', 'string'),
 ('nome_area', 'string')]

In [0]:
dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.withColumn('data_somministrazione', date_format(dati_vaccini_somministrazioni.data_somministrazione, 'yyyy-MM-dd').cast('date'))

In [0]:
dati_vaccini_somministrazioni.dtypes

Out[88]: [('data_somministrazione', 'date'),
 ('fornitore', 'string'),
 ('fascia_anagrafica', 'string'),
 ('sesso_femminile', 'int'),
 ('categoria_operatori_sanitari_sociosanitari', 'int'),
 ('categoria_personale_non_sanitario', 'int'),
 ('categoria_ospiti_rsa', 'int'),
 ('categoria_60_69', 'int'),
 ('categoria_70_79', 'int'),
 ('categoria_over80', 'int'),
 ('categoria_forze_armate', 'int'),
 ('categoria_personale_scolastico', 'int'),
 ('categoria_soggetti_fragili', 'int'),
 ('categoria_altro', 'int'),
 ('prima_dose', 'int'),
 ('seconda_dose', 'int'),
 ('codice_NUTS2', 'string'),
 ('nome_area', 'string')]

Si verifica la correttezza delle colonne categoriche.

In [0]:
num_regioni = dati_vaccini_somministrazioni.groupBy('nome_area') \
                                           .count() \
                                           .count()
print('Il numero di regioni è:', num_regioni)
print()
dati_vaccini_somministrazioni.groupBy('codice_NUTS2', 'nome_area') \
                             .count() \
                             .drop('count') \
                             .orderBy('codice_NUTS2') \
                             .show(50)

Il numero di regioni è: 21

+------------+--------------------+
codice_NUTS2| nome_area|
+------------+--------------------+
 ITC1| Piemonte|
 ITC2|Valle d'Aosta / V...|
 ITC3| Liguria|
 ITC4| Lombardia|
 ITF1| Abruzzo|
 ITF2| Molise|
 ITF3| Campania|
 ITF4| Puglia|
 ITF5| Basilicata|
 ITF6| Calabria|
 ITG1| Sicilia|
 ITG2| Sardegna|
 ITH1|Provincia Autonom...|
 ITH2|Provincia Autonom...|
 ITH3| Veneto|
 ITH4|Friuli-Venezia Gi...|
 ITH5| Emilia-Romagna|
 ITI1| Toscana|
 ITI2| Umbria|
 ITI3| Marche|
 ITI4| Lazio|
+------------+--------------------+

In [0]:
join1 = popolazione_regioni.join(dati_vaccini_somministrazioni, popolazione_regioni.denominazione_regione == dati_vaccini_somministrazioni.nome_area, how='anti') \
                    .groupBy('codice_nuts_2', 'denominazione_regione') \
                    .count() \
                    .drop('count') 

join2 = dati_vaccini_somministrazioni.join(popolazione_regioni, popolazione_regioni.denominazione_regione == dati_vaccini_somministrazioni.nome_area, how='anti') \
                    .groupBy('codice_NUTS2', 'nome_area') \
                    .count() \
                    .drop('count') 

join1.join(join2, popolazione_regioni.codice_nuts_2 == dati_vaccini_somministrazioni.codice_NUTS2).show()

dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.replace('Provincia Autonoma Trento', value='Trento', subset='nome_area')
dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.replace('Provincia Autonoma Bolzano / Bozen', value='Bolzano', subset='nome_area')
dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.replace('Friuli-Venezia Giulia', value='Friuli Venezia Giulia', subset='nome_area')
dati_vaccini_somministrazioni = dati_vaccini_somministrazioni.replace('Valle d\'Aosta / Vallée d\'Aoste', value='Valle d\'Aosta', subset='nome_area')

+-------------+---------------------+------------+--------------------+
codice_nuts_2|denominazione_regione|codice_NUTS2| nome_area|
+-------------+---------------------+------------+--------------------+
 ITC2| Valle d'Aosta| ITC2|Valle d'Aosta / V...|
 ITH2| Trento| ITH2|Provincia Autonom...|
 ITH4| Friuli Venezia Gi...| ITH4|Friuli-Venezia Gi...|
 ITH1| Bolzano| ITH1|Provincia Autonom...|
+-------------+---------------------+------------+--------------------+

In [0]:
dati_vaccini_somministrazioni.groupBy('codice_NUTS2', 'nome_area') \
                             .count() \
                             .drop('count') \
                             .orderBy('codice_NUTS2') \
                             .show(num_regioni)

+------------+--------------------+
codice_NUTS2| nome_area|
+------------+--------------------+
 ITC1| Piemonte|
 ITC2| Valle d'Aosta|
 ITC3| Liguria|
 ITC4| Lombardia|
 ITF1| Abruzzo|
 ITF2| Molise|
 ITF3| Campania|
 ITF4| Puglia|
 ITF5| Basilicata|
 ITF6| Calabria|
 ITG1| Sicilia|
 ITG2| Sardegna|
 ITH1| Bolzano|
 ITH2| Trento|
 ITH3| Veneto|
 ITH4|Friuli Venezia Gi...|
 ITH5| Emilia-Romagna|
 ITI1| Toscana|
 ITI2| Umbria|
 ITI3| Marche|
 ITI4| Lazio|
+------------+--------------------+

In [0]:
dati_vaccini_somministrazioni.groupBy('fascia_anagrafica') \
                    .count() \
                    .drop('count') \
                    .orderBy('fascia_anagrafica') \
                    .show()

+-----------------+
fascia_anagrafica|
+-----------------+
 16-19|
 20-29|
 30-39|
 40-49|
 50-59|
 60-69|
 70-79|
 80-89|
 90+|
+-----------------+

Si osserva che rispetto alla tabella popolazione_regioni manca la `fascia_anagrafica` 0-15

Si verifica la correttezza delle colonne numeriche.

In [0]:
dati_vaccini_somministrazioni.describe(['prima_dose', 'seconda_dose']) \
                             .show()

+-------+------------------+------------------+
summary| prima_dose| seconda_dose|
+-------+------------------+------------------+
 count| 42671| 42671|
 mean|323.92639028848635|135.55428745518034|
 stddev| 983.1110770051885| 545.4872184636607|
 min| 0| 0|
 max| 29495| 18906|
+-------+------------------+------------------+

In [0]:
dati_vaccini_somministrazioni.describe(['categoria_operatori_sanitari_sociosanitari', 'categoria_personale_non_sanitario', 'categoria_ospiti_rsa', 'categoria_60_69', 'categoria_70_79', 'categoria_over80', 'categoria_forze_armate', 'categoria_personale_scolastico', 'categoria_soggetti_fragili', 'categoria_altro']) \
                             .show()

+-------+------------------------------------------+---------------------------------+--------------------+-----------------+-----------------+------------------+----------------------+------------------------------+--------------------------+-----------------+
summary|categoria_operatori_sanitari_sociosanitari|categoria_personale_non_sanitario|categoria_ospiti_rsa| categoria_60_69| categoria_70_79| categoria_over80|categoria_forze_armate|categoria_personale_scolastico|categoria_soggetti_fragili| categoria_altro|
+-------+------------------------------------------+---------------------------------+--------------------+-----------------+-----------------+------------------+----------------------+------------------------------+--------------------------+-----------------+
 count| 42671| 42671| 42671| 42671| 42671| 42671| 42671| 42671| 42671| 42671|
 mean| 75.74851772866818| 20.79177895994938| 15.291064188793326|23.62534273862811|68.19699561763258|143.14276675025192| 7.570879520048745| 27.208689742448033| 69.52644653277402|8.378195964472358|
 stddev| 227.0654139987963| 66.19865795025069| 61.13419390542635| 301.257846256271|669.3949208553216| 904.6481737722518| 42.61783767939349| 151.74424110536856| 298.06499928545975|38.48846340975291|
 min| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
 max| 4662| 2303| 2316| 20021| 28029| 31050| 1214| 3583| 7448| 2008|
+-------+------------------------------------------+---------------------------------+--------------------+-----------------+-----------------+------------------+----------------------+------------------------------+--------------------------+-----------------+

Si verifica la presenza di *missing values*.

In [0]:
dati_vaccini_somministrazioni.select([count(when(isnull(c), c)).alias(c) for c in dati_vaccini_somministrazioni.drop('data').columns]).show()

+---------------------+---------+-----------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+---------+
data_somministrazione|fornitore|fascia_anagrafica|sesso_femminile|categoria_operatori_sanitari_sociosanitari|categoria_personale_non_sanitario|categoria_ospiti_rsa|categoria_60_69|categoria_70_79|categoria_over80|categoria_forze_armate|categoria_personale_scolastico|categoria_soggetti_fragili|categoria_altro|prima_dose|seconda_dose|codice_NUTS2|nome_area|
+---------------------+---------+-----------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+---------+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+---------------------+---------+-----------------+---------------+------------------------------------------+---------------------------------+--------------------+---------------+---------------+----------------+----------------------+------------------------------+--------------------------+---------------+----------+------------+------------+---------+

Si verifica la presenza di *missing values* nella serie temporale.

In [0]:
min_data = dati_vaccini_somministrazioni.agg(min('data_somministrazione').alias('minimo'))
max_data = dati_vaccini_somministrazioni.agg(max('data_somministrazione').alias('massimo'))

date = min_data.join(max_data)

date.show()

+----------+----------+
 minimo| massimo|
+----------+----------+
2020-12-27|2021-04-30|
+----------+----------+

In [0]:
time_series = date.withColumn('data_somministrazione', explode(expr('sequence(minimo, massimo, interval 1 day)'))) \
                  .select('data_somministrazione')

time_series.show()

+---------------------+
data_somministrazione|
+---------------------+
 2020-12-27|
 2020-12-28|
 2020-12-29|
 2020-12-30|
 2020-12-31|
 2021-01-01|
 2021-01-02|
 2021-01-03|
 2021-01-04|
 2021-01-05|
 2021-01-06|
 2021-01-07|
 2021-01-08|
 2021-01-09|
 2021-01-10|
 2021-01-11|
 2021-01-12|
 2021-01-13|
 2021-01-14|
 2021-01-15|
+---------------------+
only showing top 20 rows

In [0]:
date_covid = dati_vaccini_somministrazioni.select('data_somministrazione') \
                               .withColumn('data_somministrazione', date_format(dati_vaccini_somministrazioni.data_somministrazione, 'yyyy-MM-dd').cast('date')) \
                               .distinct() \
                               .orderBy('data_somministrazione')

date_covid.join(time_series, on='data_somministrazione', how='anti') \
          .show()

+---------------------+
data_somministrazione|
+---------------------+
+---------------------+

## Analisi

### Dati covid Regioni

#### Panoramica Casi per Mese

In [0]:
casi_mensili = dati_covid_regioni.select(date_format('data','yyyy-MM').alias('mese'), 'nuovi_positivi') \
                  .groupby('mese') \
                  .sum('nuovi_positivi') \
                  .withColumnRenamed('sum(nuovi_positivi)', 'casi')

In [0]:
tamponi_mensili = dati_covid_regioni.select(date_format('data','yyyy-MM').alias('mese'), 'denominazione_regione', 'tamponi') \
                                    .groupby('mese', 'denominazione_regione') \
                                    .max('tamponi') \
                                    .groupby('mese') \
                                    .sum('max(tamponi)') \
                                    .withColumnRenamed('sum(max(tamponi))', 'tamponi')

my_window = Window.partitionBy().orderBy("mese") 

tamponi_mensili = tamponi_mensili.withColumn("prev_value", lag(tamponi_mensili.tamponi) \
                                 .over(my_window)) 
tamponi_mensili = tamponi_mensili.withColumn("diff", when(isnull(tamponi_mensili.tamponi - tamponi_mensili.prev_value), tamponi_mensili.tamponi).otherwise(tamponi_mensili.tamponi - tamponi_mensili.prev_value)) \
                                 .drop('tamponi', 'prev_value') \
                                 .withColumnRenamed('diff', 'tamponi')

In [0]:
panoramica_mensile = casi_mensili.join(tamponi_mensili, on='mese') \
                                 .orderBy('mese', ascending=False)

panoramica_mensile = panoramica_mensile.withColumn('percentuale_tamponi_positivi', panoramica_mensile.casi / panoramica_mensile.tamponi * 100)

panoramica_mensile.show()

# path = "dbfs:/FileStore/gu.dichiara@studenti.unina.it/panoramica_mensile.csv"
# panoramica_mensile.write.csv(path, mode='overwrite', sep=',', header=True)

+-------+------+-------+----------------------------+
 mese| casi|tamponi|percentuale_tamponi_positivi|
+-------+------+-------+----------------------------+
2021-04|439446|8677255| 5.064343504944825|
2021-03|648200|9769610| 6.634860552263601|
2021-02|372503|7466161| 4.989217350121435|
2021-01|445585|5948709| 7.4904487679595695|
2020-12|508914|4772918| 10.662533904835575|
2020-11|922206|6160638| 14.969326228874348|
2020-10|364607|4450539| 8.192423434554781|
2020-09| 45623|2689063| 1.6966132812805055|
2020-08| 21702|1831746| 1.1847712510358968|
2020-07| 7006|1423003| 0.4923390885331935|
2020-06| 7584|1511371| 0.5017960513996894|
2020-05| 27556|1899522| 1.4506807502097896|
2020-04| 99671|1472249| 6.769982523336745|
2020-03|104664| 488307| 21.434056853577772|
2020-02| 1120| 18661| 6.001821981673007|
+-------+------+-------+----------------------------+

#### Attuali Positivi per Regione

In [0]:
attuali_positivi_regione = dati_covid_regioni.where(col("data") >= current_date() - expr("INTERVAL 1 days")) \
                                             .select('data','denominazione_regione', 'totale_positivi') \
                                             .withColumnRenamed('denominazione_regione', 'regione') \
                                             .withColumnRenamed('totale_positivi', 'attuali_positivi')

popolazione = popolazione_regioni.groupBy('denominazione_regione') \
                                 .sum('totale_generale')

cond = [popolazione.denominazione_regione == attuali_positivi_regione.regione]
attuali_positivi_regione = attuali_positivi_regione.join(popolazione, cond, how='left') \
                                                   .drop('denominazione_regione') \
                                                   .withColumnRenamed('sum(totale_generale)', 'popolazione') \
                                                   .orderBy('regione')

attuali_positivi_regione = attuali_positivi_regione.withColumn('positivi_per_100k', (attuali_positivi_regione.attuali_positivi / attuali_positivi_regione.popolazione * 100000).cast('int')) \
                                                   .orderBy('positivi_per_100k', ascending=False)

attuali_positivi_regione.show(21)

+----------+--------------------+----------------+-----------+-----------------+
 data| regione|attuali_positivi|popolazione|positivi_per_100k|
+----------+--------------------+----------------+-----------+-----------------+
2021-04-29| Campania| 91885| 5712143| 1608|
2021-04-30| Campania| 91388| 5712143| 1599|
2021-04-29| Puglia| 48429| 3953305| 1225|
2021-04-30| Puglia| 47592| 3953305| 1203|
2021-04-30| Basilicata| 6230| 553254| 1126|
2021-04-29| Basilicata| 6160| 553254| 1113|
2021-04-29| Sardegna| 17235| 1611621| 1069|
2021-04-30| Sardegna| 17093| 1611621| 1060|
2021-04-30| Emilia-Romagna| 44356| 4464119| 993|
2021-04-29| Emilia-Romagna| 43651| 4464119| 977|
2021-04-30| Calabria| 15214| 1894110| 803|
2021-04-29| Calabria| 14952| 1894110| 789|
2021-04-29| Lazio| 45362| 5755700| 788|
2021-04-30| Lazio| 44696| 5755700| 776|
2021-04-29| Abruzzo| 8551| 1293941| 660|
2021-04-30| Abruzzo| 8383| 1293941| 647|
2021-04-29|Friuli Venezia Gi...| 7627| 1206216| 632|
2021-04-30|Friuli Venezia Gi...| 7578| 1206216| 628|
2021-04-29| Valle d'Aosta| 764| 125034| 611|
2021-04-30| Valle d'Aosta| 749| 125034| 599|
2021-04-29| Toscana| 21127| 3692555| 572|
+----------+--------------------+----------------+-----------+-----------------+
only showing top 21 rows

#### Dati Critici per Mese

In [0]:
decessi_mensili = dati_covid_regioni.select(date_format('data','yyyy-MM').alias('mese'), 'denominazione_regione', 'deceduti') \
                                    .groupby('mese', 'denominazione_regione') \
                                    .max('deceduti') \
                                    .groupby('mese') \
                                    .sum('max(deceduti)') \
                                    .withColumnRenamed('sum(max(deceduti))', 'deceduti')

my_window = Window.partitionBy().orderBy("mese") 

decessi_mensili = decessi_mensili.withColumn("prev_value", lag(decessi_mensili.deceduti) \
                                 .over(my_window)) 
decessi_mensili = decessi_mensili.withColumn("diff", when(isnull(decessi_mensili.deceduti - decessi_mensili.prev_value), decessi_mensili.deceduti).otherwise(decessi_mensili.deceduti - decessi_mensili.prev_value)) \
                                 .drop('deceduti', 'prev_value') \
                                 .withColumnRenamed('diff', 'deceduti')

In [0]:
terapie_intensive_mensili = dati_covid_regioni.select('data', 'terapia_intensiva') \
                  .groupby('data') \
                  .sum('terapia_intensiva') \
                  .select(date_format('data','yyyy-MM').alias('mese'), 'sum(terapia_intensiva)') \
                  .groupby('mese') \
                  .mean('sum(terapia_intensiva)') \
                  .withColumnRenamed('avg(sum(terapia_intensiva))', 'media_terapia_intensiva')

terapie_intensive_mensili = terapie_intensive_mensili.withColumn('media_terapia_intensiva', terapie_intensive_mensili.media_terapia_intensiva.cast('int'))

In [0]:
dati_critici_mensili = decessi_mensili.join(casi_mensili, on='mese') \
                                      .join(terapie_intensive_mensili, on='mese') \
                                      .orderBy('mese', ascending=False)

dati_critici_mensili.show()

+-------+--------+------+-----------------------+
 mese|deceduti| casi|media_terapia_intensiva|
+-------+--------+------+-----------------------+
2021-04| 11461|439446| 3306|
2021-03| 11647|648200| 3127|
2021-02| 9183|372503| 2128|
2021-01| 14357|445585| 2483|
2020-12| 18583|508914| 3005|
2020-11| 16958|922206| 3231|
2020-10| 2724|364607| 785|
2020-09| 411| 45623| 194|
2020-08| 341| 21702| 58|
2020-07| 311| 7006| 57|
2020-06| 1406| 7584| 207|
2020-05| 5458| 27556| 882|
2020-04| 15539| 99671| 2975|
2020-03| 12399|104664| 1985|
2020-02| 29| 1120| 53|
+-------+--------+------+-----------------------+

### Dati Somministrazioni Vaccini

#### Percentuale Popolazione Vaccinata per Regione
Si riporta la percentuale di popolazione vaccinata (prima e seconda dose) per regione.

In [0]:
dosi_somministrate_regione = dati_vaccini_somministrazioni.groupBy('nome_area') \
                                                          .sum('prima_dose', 'seconda_dose') \
                                                          .orderBy('nome_area') \
                                                          .withColumnRenamed('nome_area', 'regione') \
                                                          .withColumnRenamed('sum(prima_dose)', 'vaccinati_prima_dose') \
                                                          .withColumnRenamed('sum(seconda_dose)', 'vaccinati_seconda_dose')

popolazione = popolazione_regioni.groupBy('denominazione_regione').sum('totale_generale')

cond = [popolazione.denominazione_regione == dosi_somministrate_regione.regione]
dosi_somministrate_regione = dosi_somministrate_regione.join(popolazione, cond, how='left') \
                                                       .drop('denominazione_regione') \
                                                       .withColumnRenamed('sum(totale_generale)', 'popolazione') \
                                                       .orderBy('regione')

dosi_somministrate_regione = dosi_somministrate_regione.withColumn('percentuale_prima_dose', dosi_somministrate_regione.vaccinati_prima_dose / dosi_somministrate_regione.popolazione * 100)
dosi_somministrate_regione = dosi_somministrate_regione.withColumn('percentuale_seconda_dose', dosi_somministrate_regione.vaccinati_seconda_dose / dosi_somministrate_regione.popolazione * 100)

dosi_somministrate_regione.show(21)

+--------------------+--------------------+----------------------+-----------+----------------------+------------------------+
 regione|vaccinati_prima_dose|vaccinati_seconda_dose|popolazione|percentuale_prima_dose|percentuale_seconda_dose|
+--------------------+--------------------+----------------------+-----------+----------------------+------------------------+
 Abruzzo| 311427| 122447| 1293941| 24.068098931867837| 9.463105350243945|
 Basilicata| 128334| 53415| 553254| 23.196217288984805| 9.65469748072314|
 Bolzano| 136324| 51672| 532644| 25.593830025307714| 9.701038592380652|
 Calabria| 371109| 158541| 1894110| 19.592790281451446| 8.370210811410109|
 Campania| 1217410| 470116| 5712143| 21.312666717202283| 8.230116087780015|
 Emilia-Romagna| 1094522| 521458| 4464119| 24.518208407974786| 11.68109541882732|
Friuli Venezia Gi...| 281012| 132954| 1206216| 23.29698826744132| 11.022403947551682|
 Lazio| 1308950| 582981| 5755700| 22.741803777125284| 10.128759316851122|
 Liguria| 418811| 183623| 1524826| 27.466150236158093| 12.042226457313818|
 Lombardia| 2357480| 939301| 10027602| 23.509907952070694| 9.36715477937796|
 Marche| 396866| 147736| 1512672| 26.236090837934462| 9.7665587781092|
 Molise| 74734| 39041| 300516| 24.868559411146162| 12.991321593525804|
 Piemonte| 1076458| 454229| 4311217| 24.968773318531635| 10.535980907479257|
 Puglia| 953086| 310022| 3953305| 24.108587624784832| 7.842096676072299|
 Sardegna| 350304| 150663| 1611621| 21.736127786868003| 9.348537900660267|
 Sicilia| 969999| 439306| 4875290| 19.89623181390235| 9.010869097017817|
 Toscana| 864646| 376214| 3692555| 23.415927454025738| 10.188446752993524|
 Trento| 132575| 41997| 545425| 24.306733281386077| 7.69986707613329|
 Umbria| 214573| 87974| 870165| 24.658886533013856| 10.110036602253595|
 Valle d'Aosta| 32706| 12262| 125034| 26.157685109650174| 9.806932514356095|
 Veneto| 1130937| 508285| 4879133| 23.17905660698325| 10.417527048350598|
+--------------------+--------------------+----------------------+-----------+----------------------+------------------------+

#### Percentuale Popolazione Vaccinata per Fascia Anagrafica
Si riporta la percentuale di popolazione vaccinata (prima e seconda dose) per fascia anagrafica.

In [0]:
dosi_somministrate_fascia = dati_vaccini_somministrazioni.groupBy('fascia_anagrafica') \
                                                          .sum('prima_dose', 'seconda_dose') \
                                                          .orderBy('fascia_anagrafica') \
                                                          .withColumnRenamed('sum(prima_dose)', 'vaccinati_prima_dose') \
                                                          .withColumnRenamed('sum(seconda_dose)', 'vaccinati_seconda_dose')

popolazione = popolazione_regioni.groupBy('range_eta').sum('totale_generale')

cond = [popolazione.range_eta == dosi_somministrate_fascia.fascia_anagrafica]
dosi_somministrate_fascia = dosi_somministrate_fascia.join(popolazione, cond, how='left') \
                                                       .withColumnRenamed('sum(totale_generale)', 'popolazione') \
                                                       .drop('range_eta') \
                                                       .orderBy('fascia_anagrafica')

dosi_somministrate_fascia = dosi_somministrate_fascia.withColumn('percentuale_prima_dose', dosi_somministrate_fascia.vaccinati_prima_dose / dosi_somministrate_fascia.popolazione * 100)
dosi_somministrate_fascia = dosi_somministrate_fascia.withColumn('percentuale_seconda_dose', dosi_somministrate_fascia.vaccinati_seconda_dose / dosi_somministrate_fascia.popolazione * 100)

dosi_somministrate_fascia.show(9)

+-----------------+--------------------+----------------------+-----------+----------------------+------------------------+
fascia_anagrafica|vaccinati_prima_dose|vaccinati_seconda_dose|popolazione|percentuale_prima_dose|percentuale_seconda_dose|
+-----------------+--------------------+----------------------+-----------+----------------------+------------------------+
 16-19| 27109| 8974| 2298846| 1.1792438466952548| 0.3903697768358559|
 20-29| 561462| 283521| 6084382| 9.227921586777423| 4.659815902420328|
 30-39| 813228| 378835| 6854632| 11.863919171736717| 5.526700776934487|
 40-49| 1191640| 488453| 8937229| 13.333439257291047| 5.465374110924091|
 50-59| 1650866| 668824| 9414195| 17.535923145845185| 7.104420505417616|
 60-69| 2200593| 557865| 7364364| 29.881643547222815| 7.575195902864117|
 70-79| 3527048| 514168| 5968373| 59.09563628144555| 8.61487711977787|
 80-89| 3147477| 2368033| 3628160| 86.75132849708943| 65.26815245193156|
 90+| 702840| 515564| 791543| 88.79365997804288| 65.13404830817782|
+-----------------+--------------------+----------------------+-----------+----------------------+------------------------+

#### Percentuale Popolazione Vaccinata per Regione e Fascia Anagrafica
Si riporta la percentuale di popolazione vaccinata (prima e seconda dose) per regione e fascia anagrafica.

In [0]:
dosi_somministrate = dati_vaccini_somministrazioni.groupBy('nome_area', 'fascia_anagrafica') \
                                                  .sum('prima_dose', 'seconda_dose') \
                                                  .orderBy('nome_area', 'fascia_anagrafica') \
                                                  .withColumnRenamed('nome_area', 'regione') \
                                                  .withColumnRenamed('sum(prima_dose)', 'vaccinati_prima_dose') \
                                                  .withColumnRenamed('sum(seconda_dose)', 'vaccinati_seconda_dose')

cond = [popolazione_regioni.denominazione_regione == dosi_somministrate.regione, popolazione_regioni.range_eta == dosi_somministrate.fascia_anagrafica]
dosi_somministrate = dosi_somministrate.join(popolazione_regioni.select('denominazione_regione', 'range_eta', 'totale_generale'), cond, how='left') \
                                       .drop('denominazione_regione', 'range_eta') \
                                       .withColumnRenamed('totale_generale', 'popolazione') \
                                       .orderBy('regione', 'fascia_anagrafica')

dosi_somministrate = dosi_somministrate.withColumn('percentuale_prima_dose', dosi_somministrate.vaccinati_prima_dose / dosi_somministrate.popolazione * 100)
dosi_somministrate = dosi_somministrate.withColumn('percentuale_seconda_dose', dosi_somministrate.vaccinati_seconda_dose / dosi_somministrate.popolazione * 100)

dosi_somministrate.show(189)

#### Categorie

In [0]:
dosi_somministrate_categorie_mensili = dati_vaccini_somministrazioni.select(date_format('data_somministrazione','yyyy-MM').alias('mese'), \
                                                                            'categoria_operatori_sanitari_sociosanitari', 'categoria_personale_non_sanitario', 'categoria_ospiti_rsa', \
                                                                            'categoria_60_69', 'categoria_70_79', 'categoria_over80', 'categoria_forze_armate', 'categoria_personale_scolastico', \
                                                                            'categoria_soggetti_fragili', 'categoria_altro') \
                                                                    .groupby('mese') \
                                                                    .agg(sum('categoria_operatori_sanitari_sociosanitari').alias('operatori_sanitari_sociosanitari'), \
                                                                         sum('categoria_personale_non_sanitario').alias('personale_non_sanitario'), \
                                                                         sum('categoria_ospiti_rsa').alias('ospiti_rsa'), \
                                                                         sum('categoria_60_69').alias('60_69'), \
                                                                         sum('categoria_70_79').alias('70_79'), \
                                                                         sum('categoria_over80').alias('over80'), \
                                                                         sum('categoria_forze_armate').alias('forze_armate'), \
                                                                         sum('categoria_personale_scolastico').alias('personale_scolastico'), \
                                                                         sum('categoria_soggetti_fragili').alias('soggetti_fragili'), \
                                                                         sum('categoria_altro').alias('altro')) \
                                                                      .orderBy('mese')

dosi_somministrate_categorie_mensili.show()

+-------+--------------------------------+-----------------------+----------+------+-------+-------+------------+--------------------+----------------+------+
 mese|operatori_sanitari_sociosanitari|personale_non_sanitario|ospiti_rsa| 60_69| 70_79| over80|forze_armate|personale_scolastico|soggetti_fragili| altro|
+-------+--------------------------------+-----------------------+----------+------+-------+-------+------------+--------------------+----------------+------+
2020-12| 33952| 1493| 3262| 163| 42| 51| 0| 8| 1| 843|
2021-01| 1458495| 229719| 200883| 11933| 4473| 17782| 177| 322| 142| 55551|
2021-02| 901164| 269858| 212559| 14415| 8363| 639845| 67405| 176823| 2618| 76433|
2021-03| 598546| 280802| 154531| 86847| 401469|2823275| 215292| 847361| 438984|116396|
2021-04| 240108| 105334| 81250|894759|2495687|2627092| 40183| 136508| 2525018|108283|
+-------+--------------------------------+-----------------------+----------+------+-------+-------+------------+--------------------+----------------+------+

In [0]:
dosi_somministrate_mensili = dati_vaccini_somministrazioni.select(date_format('data_somministrazione','yyyy-MM').alias('mese'), 'prima_dose', 'seconda_dose') \
                                                          .groupby('mese') \
                                                          .sum('prima_dose', 'seconda_dose')

dosi_somministrate_mensili = dosi_somministrate_mensili.withColumnRenamed('sum(prima_dose)', 'vaccinati_prima_dose') \
                                                       .withColumnRenamed('sum(seconda_dose)', 'vaccinati_seconda_dose')


totale_dosi_somministrate_mensili = dosi_somministrate_mensili.withColumn('totale_dosi', dosi_somministrate_mensili.vaccinati_prima_dose + dosi_somministrate_mensili.vaccinati_seconda_dose)

percentuale_dosi_categorie = totale_dosi_somministrate_mensili.join(dosi_somministrate_categorie_mensili, on='mese') \
                                                              .drop('vaccinati_prima_dose', 'vaccinati_seconda_dose') \
                                                              .withColumn('operatori_sanitari_sociosanitari', round(dosi_somministrate_categorie_mensili.operatori_sanitari_sociosanitari / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('personale_non_sanitario', round(dosi_somministrate_categorie_mensili.personale_non_sanitario / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('ospiti_rsa', round(dosi_somministrate_categorie_mensili.ospiti_rsa / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('60_69', round(dosi_somministrate_categorie_mensili['60_69'] / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('70_79', round(dosi_somministrate_categorie_mensili['70_79'] / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('over80', round(dosi_somministrate_categorie_mensili.over80 / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('forze_armate', round(dosi_somministrate_categorie_mensili.forze_armate / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('personale_scolastico', round(dosi_somministrate_categorie_mensili.personale_scolastico / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('soggetti_fragili', round(dosi_somministrate_categorie_mensili.soggetti_fragili / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .withColumn('altro', round(dosi_somministrate_categorie_mensili.altro / totale_dosi_somministrate_mensili.totale_dosi *100, 3)) \
                                                              .drop('totale_dosi') \
                                                              .orderBy('mese')
    
percentuale_dosi_categorie.show()

+-------+--------------------------------+-----------------------+----------+-----+------+------+------------+--------------------+----------------+-----+
 mese|operatori_sanitari_sociosanitari|personale_non_sanitario|ospiti_rsa|60_69| 70_79|over80|forze_armate|personale_scolastico|soggetti_fragili|altro|
+-------+--------------------------------+-----------------------+----------+-----+------+------+------------+--------------------+----------------+-----+
2020-12| 85.274| 3.75| 8.193|0.409| 0.105| 0.128| 0.0| 0.02| 0.003|2.117|
2021-01| 73.681| 11.605| 10.148|0.603| 0.226| 0.898| 0.009| 0.016| 0.007|2.806|
2021-02| 38.032| 11.389| 8.971|0.608| 0.353|27.004| 2.845| 7.463| 0.11|3.226|
2021-03| 10.037| 4.709| 2.591|1.456| 6.732|47.343| 3.61| 14.209| 7.361|1.952|
2021-04| 2.595| 1.138| 0.878|9.669|26.968|28.388| 0.434| 1.475| 27.285| 1.17|
+-------+--------------------------------+-----------------------+----------+-----+------+------+------------+--------------------+----------------+-----+

#### Fornitori e Fasce Anagrafiche

In [0]:
dosi_fascia_fornitore = dati_vaccini_somministrazioni.groupBy('fascia_anagrafica', 'fornitore') \
                                                          .sum('prima_dose', 'seconda_dose') \
                                                          .orderBy('fascia_anagrafica', 'fornitore') 

totale_dosi_somministrate_fascia = dosi_somministrate_fascia.select('fascia_anagrafica', 'vaccinati_prima_dose', 'vaccinati_seconda_dose') \
                                                            .withColumn('totale_dosi', dosi_somministrate_fascia.vaccinati_prima_dose + dosi_somministrate_fascia.vaccinati_seconda_dose) \
                                                            .drop('vaccinati_prima_dose', 'vaccinati_seconda_dose')

dosi_fascia_fornitore = dosi_fascia_fornitore.join(totale_dosi_somministrate_fascia, on='fascia_anagrafica')

dosi_fascia_fornitore = dosi_fascia_fornitore.withColumn('percentuale_dosi', round((dosi_fascia_fornitore['sum(prima_dose)'] + dosi_fascia_fornitore['sum(seconda_dose)']) *100 /  dosi_fascia_fornitore.totale_dosi, 3)) \
                                             .drop('sum(prima_dose)', 'sum(seconda_dose)', 'totale_dosi') \
                                             .orderBy('fascia_anagrafica', 'fornitore')

dosi_fascia_fornitore.show(100)

+-----------------+--------------------+----------------+
fascia_anagrafica| fornitore|percentuale_dosi|
+-----------------+--------------------+----------------+
 16-19| Janssen| 0.022|
 16-19| Moderna| 7.416|
 16-19| Pfizer/BioNTech| 85.6|
 16-19|Vaxzevria (AstraZ...| 6.962|
 20-29| Janssen| 0.117|
 20-29| Moderna| 4.448|
 20-29| Pfizer/BioNTech| 74.639|
 20-29|Vaxzevria (AstraZ...| 20.796|
 30-39| Janssen| 0.123|
 30-39| Moderna| 4.421|
 30-39| Pfizer/BioNTech| 70.218|
 30-39|Vaxzevria (AstraZ...| 25.237|
 40-49| Janssen| 0.097|
 40-49| Moderna| 4.752|
 40-49| Pfizer/BioNTech| 66.897|
 40-49|Vaxzevria (AstraZ...| 28.254|
 50-59| Janssen| 0.07|
 50-59| Moderna| 5.21|
 50-59| Pfizer/BioNTech| 69.307|
 50-59|Vaxzevria (AstraZ...| 25.414|
 60-69| Janssen| 0.592|
 60-69| Moderna| 6.35|
 60-69| Pfizer/BioNTech| 61.809|
 60-69|Vaxzevria (AstraZ...| 31.249|
 70-79| Janssen| 0.659|
 70-79| Moderna| 7.109|
 70-79| Pfizer/BioNTech| 48.555|
 70-79|Vaxzevria (AstraZ...| 43.677|
 80-89| Janssen| 0.009|
 80-89| Moderna| 9.487|
 80-89| Pfizer/BioNTech| 89.357|
 80-89|Vaxzevria (AstraZ...| 1.147|
 90+| Janssen| 0.006|
 90+| Moderna| 13.703|
 90+| Pfizer/BioNTech| 85.609|
 90+|Vaxzevria (AstraZ...| 0.682|
+-----------------+--------------------+----------------+

### Correlazione Dati

#### Dosi Somministrate e Media terapie intensive

In [0]:
dosi_somministrate_terapie_intensive = dosi_somministrate_mensili.join(terapie_intensive_mensili, on='mese', how='left') \
                                                                 .orderBy('mese', ascending=False)


dosi_somministrate_terapie_intensive.show()

+-------+--------------------+----------------------+-----------------------+
 mese|vaccinati_prima_dose|vaccinati_seconda_dose|media_terapia_intensiva|
+-------+--------------------+----------------------+-----------------------+
2021-04| 6706657| 2547565| 3306|
2021-03| 4148302| 1815201| 3127|
2021-02| 1588324| 781159| 2128|
2021-01| 1339165| 640312| 2483|
2020-12| 39815| 0| 3005|
+-------+--------------------+----------------------+-----------------------+

#### Dosi Somministrate e Media terapie intensive

In [0]:
attuali_terapie_intensive_regioni = dati_covid_regioni.where(col("data") >= current_date() - expr("INTERVAL 1 days")) \
                                             .select('data', 'denominazione_regione', 'terapia_intensiva') \
                                             .withColumnRenamed('denominazione_regione', 'regione')

In [0]:
attuali_dosi_somministrate_terapie_intensive = attuali_terapie_intensive_regioni.join(dosi_somministrate_regione, on='regione') \
                                                                                .select('data', 'regione', 'percentuale_prima_dose', 'percentuale_seconda_dose', 'terapia_intensiva') \
                                                                                .orderBy('regione')

attuali_dosi_somministrate_terapie_intensive.show(21)

+----------+--------------------+----------------------+------------------------+-----------------+
 data| regione|percentuale_prima_dose|percentuale_seconda_dose|terapia_intensiva|
+----------+--------------------+----------------------+------------------------+-----------------+
2021-04-30| Abruzzo| 24.068098931867837| 9.463105350243945| 36|
2021-04-29| Abruzzo| 24.068098931867837| 9.463105350243945| 39|
2021-04-30| Basilicata| 23.196217288984805| 9.65469748072314| 8|
2021-04-29| Basilicata| 23.196217288984805| 9.65469748072314| 6|
2021-04-30| Bolzano| 25.593830025307714| 9.701038592380652| 2|
2021-04-29| Bolzano| 25.593830025307714| 9.701038592380652| 3|
2021-04-29| Calabria| 19.592790281451446| 8.370210811410109| 48|
2021-04-30| Calabria| 19.592790281451446| 8.370210811410109| 46|
2021-04-30| Campania| 21.312666717202283| 8.230116087780015| 137|
2021-04-29| Campania| 21.312666717202283| 8.230116087780015| 137|
2021-04-29| Emilia-Romagna| 24.518208407974786| 11.68109541882732| 234|
2021-04-30| Emilia-Romagna| 24.518208407974786| 11.68109541882732| 226|
2021-04-30|Friuli Venezia Gi...| 23.29698826744132| 11.022403947551682| 30|
2021-04-29|Friuli Venezia Gi...| 23.29698826744132| 11.022403947551682| 30|
2021-04-30| Lazio| 22.741803777125284| 10.128759316851122| 284|
2021-04-29| Lazio| 22.741803777125284| 10.128759316851122| 291|
2021-04-29| Liguria| 27.466150236158093| 12.042226457313818| 67|
2021-04-30| Liguria| 27.466150236158093| 12.042226457313818| 65|
2021-04-30| Lombardia| 23.509907952070694| 9.36715477937796| 550|
2021-04-29| Lombardia| 23.509907952070694| 9.36715477937796| 557|
2021-04-30| Marche| 26.236090837934462| 9.7665587781092| 67|
+----------+--------------------+----------------------+------------------------+-----------------+
only showing top 21 rows

# IDEE
* Dubito che metteremo mano, ma ci sono somministrazioni di vaccini a singola dose.